# RAG Retrieval Optimization - Query Rewriting using Amazon Bedrock and Llamaindex

Query rewrite in RAG is a technique that reformulates or transforms the original user query to improve the retrieval process and ultimately enhance the quality of generated responses. This strategy involves modifying the input query in various ways, such as expanding it with related terms, simplifying complex queries, or breaking them down into sub-questions. The goal is to bridge the gap between the user's natural language input and the system's ability to find relevant information in the knowledge base. By rewriting queries, RAG systems can increase both recall (retrieving more relevant documents) and precision (improving the relevance of retrieved information), leading to more accurate and comprehensive answers.

In this lab, we will build a query engine using LlamaIndex to answering a complex query about Amazon's 10K SEC filing from years 2022 and 2023. The lab uses the SubQuestionQueryEngine module from LLamaIndex to first breaks down the complex query into sub questions for each relevant data source then gathers all the intermediate responses and synthesizes to get better final response from Amazon's 10K documents.

Here are the components we used:

- Vector Database (Faiss / local)
- LLM (Amazon Bedrock - Claude3 Sonnet)
- Embeddings Model (Bedrock Titan Text Embeddings v2.0)
- Datasets ( Amazons 10-k sec filings from year 2022 and 2023 )
- Llamaindex  (This example is built on referece llamaindex documentation available at - https://docs.llamaindex.ai/en/stable/examples/query_engine/sub_question_query_engine/)


## Pre-req
You must run the `[workshop_setup.ipynb]`(../lab00-setup/workshop_setup.ipynb) notebook in `lab00-setup` before starting this lab.

In [1]:
import warnings
warnings.warn("Warning: if you did not run lab00-setup, please go back and run the lab00 notebook") 

/tmp/ipykernel_4493/952814476.py:2: UserWarning: Warning: if you did not run lab00-setup, please go back and run the lab00 notebook
  warnings.warn("Warning: if you did not run lab00-setup, please go back and run the lab00 notebook")


### > Setup
We start by importing necessary llamaindex libraries

In [2]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.core import Settings
from termcolor import colored

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /opt/conda/lib/python3.11/site-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


We select Anthropic Claude3 Sonnet as our LLM. For embedding model, we are selecting Amazon Titan Text Embed v2.0. Chunk size is set at 512 for this example.

In [3]:
import json
from typing import Sequence, List
from llama_index.core.settings import Settings
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding, Models

# define the LLM
llm = Bedrock(model = "anthropic.claude-3-sonnet-20240229-v1:0")

# define the embedding model
embed_model = BedrockEmbedding(model = "amazon.titan-embed-text-v2:0")

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
import nest_asyncio
nest_asyncio.apply()

### > Document Ingestion
We ingest the documents and use [Titan Text Embeddings v2.0 model](https://docs.aws.amazon.com/bedrock/latest/userguide/titan-embedding-models.html) to create the embedding for each document chunk. The amazon folder has SEC-10k files from 2022 and 2023.

In [4]:
# load data
amazon_secfiles = SimpleDirectoryReader(input_dir="../data/lab03/amazon/").load_data()

vector_index = VectorStoreIndex.from_documents(
    amazon_secfiles,
    use_async=True,
)

Define the query engine from index

In [5]:
# build index and query engine
vector_query_engine = vector_index.as_query_engine(
    top_k=1
)

### > Test Using Naive RAG
Let's try a common multi-part question on the 10K document using the naive RAG approach...

Example questions
- What are amazons key priorities before, during and after covid?
- What were key challenges faced by Amazon in year 2022 and 2023?

In [6]:
query = "What are amazons key priorities before, during and after covid?"

In [7]:
raw_response = vector_query_engine.query(query)
print(colored(raw_response, "green"))

Based on the context provided, Amazon's key priorities before, during, and after COVID-19 seem to be focused on maintaining operational resilience and minimizing disruptions to its business. The context highlights several risks and challenges that Amazon faces, including:

1. Disruptions from natural or human-caused disasters, public health crises (like COVID-19), geopolitical events, labor disputes, and other similar events that could impact Amazon's operations, customers, sellers, and suppliers.

2. Potential negative impacts of climate change, such as increased operating costs due to extreme weather events, increased investment requirements for transitioning to a low-carbon economy, decreased demand for products and services due to changes in customer behavior, increased compliance costs, and reputational damage.

3. Optimizing and operating its fulfillment network and data centers effectively to meet customer demand and avoid excess or insufficient capacity, service interruptions, 

### > Test Using SubQuestionQueryEngine to rewrite the query

`SubQuestionQueryEngine` is a llamaindex module designed to tackle the problem of answering a complex query using multiple data sources.
It first breaks down the complex query into sub questions for each relevant data source, then gather all the intermediate reponses and synthesizes a final response.

In this example, the `SubQuestionQueryEngine` will use the LLM model (Claude3 Sonnet) to breaking down of complex queries into sub-queries.

In [8]:
# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine,
        metadata=ToolMetadata(
            name="Amazon-10k",
            description="Amazon SEC 10-k filings for years 2022 and 2023",
        ),
    ),
]

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    use_async=True,
)

Run the same query using `SubQuestionQueryEngine`

In [9]:
transformed_response = query_engine.query(query)
print("\n")
print(colored(transformed_response, "green"))

Generated 3 sub questions.
[Amazon-10k] Q: What were Amazon's key priorities before the COVID-19 pandemic?
[Amazon-10k] A: Based on the context provided, Amazon's key priorities before the COVID-19 pandemic appear to have been maintaining their unique culture of innovation, customer obsession, and long-term thinking, which the context states has been critical to their growth and success. The context also highlights optimizing and operating their fulfillment network and data centers effectively as a key priority to avoid excess or insufficient capacity, service interruptions, and increased costs. Additionally, managing various risks and factors that could impact their operations and expenses, such as transportation costs, labor markets, technology trends, and potential disruptions from natural disasters or geopolitical events, seem to have been important priorities for Amazon.
[Amazon-10k] Q: How did Amazon's priorities shift during the COVID-19 pandemic?
[Amazon-10k] A: The context inf

### > Display the results side-by-side 

Notice splitting the question into sub questions increase your chances of matching the right and complete information and generate a more comprehensive final answer.

In [10]:
import pandas as pd
from IPython.display import display, HTML

# Create the first table
df = pd.DataFrame({
    'Naive RAG': [query, raw_response],
    'RAG w/ Query Rewrite': [query, transformed_response]
})

output=""
output += df.style.hide().set_table_attributes("style='display:inline'")._repr_html_()
output += "&nbsp;"

display(HTML(output))

Naive RAG,RAG w/ Query Rewrite
"What are amazons key priorities before, during and after covid?","What are amazons key priorities before, during and after covid?"
"Based on the context provided, Amazon's key priorities before, during, and after COVID-19 seem to be focused on maintaining operational resilience and minimizing disruptions to its business. The context highlights several risks and challenges that Amazon faces, including: 1. Disruptions from natural or human-caused disasters, public health crises (like COVID-19), geopolitical events, labor disputes, and other similar events that could impact Amazon's operations, customers, sellers, and suppliers. 2. Potential negative impacts of climate change, such as increased operating costs due to extreme weather events, increased investment requirements for transitioning to a low-carbon economy, decreased demand for products and services due to changes in customer behavior, increased compliance costs, and reputational damage. 3. Optimizing and operating its fulfillment network and data centers effectively to meet customer demand and avoid excess or insufficient capacity, service interruptions, and increased costs. 4. Retaining key executives, employees, and managing its workforce effectively, including addressing potential labor union efforts. 5. Managing supplier relationships and ensuring availability of merchandise, content, components, and services from significant suppliers, including those with limited or single sources of supply. While not explicitly stated, these priorities likely remained crucial for Amazon before, during, and after the COVID-19 pandemic to maintain business continuity, adapt to changing circumstances, and mitigate potential disruptions to its operations and supply chain.","Before the COVID-19 pandemic, Amazon's key priorities appear to have been maintaining their culture of innovation, customer obsession, and long-term thinking, as well as optimizing their fulfillment network and data centers to meet demand efficiently while managing various operational risks and factors. During the pandemic, Amazon likely had to shift priorities to address disruptions caused by the public health crisis, such as fluctuations in customer demand patterns, potential fulfillment capacity constraints, and staffing challenges in fulfillment centers and customer service operations. After the COVID-19 pandemic, some of Amazon's key priorities seem to be enhancing operational resilience to handle variability from factors like seasonality, promotions, and unforeseeable events; managing climate-related risks and transitioning to a low-carbon economy; and mitigating potential disruptions from natural disasters, geopolitical events, and restrictive governmental actions, particularly in regions with significant operations."
